## Deliverable 2. Create a Customer Travel Destinations Map.

In [3]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Rikitea,PF,-23.1203,-134.9692,73.94,80,100,22.59,overcast clouds
1,1,Kapaa,US,22.0752,-159.3190,82.38,74,90,4.00,overcast clouds
2,2,Hilo,US,19.7297,-155.0900,80.33,69,20,4.61,few clouds
3,3,Waipawa,NZ,-41.4122,175.5153,53.46,68,0,14.83,clear sky
4,4,Airai,TL,-8.9266,125.4092,56.84,83,34,1.77,scattered clouds


In [6]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))


What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [7]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Kapaa,US,22.0752,-159.3190,82.38,74,90,4.00,overcast clouds
2,2,Hilo,US,19.7297,-155.0900,80.33,69,20,4.61,few clouds
5,5,Atuona,PF,-9.8000,-139.0333,77.49,78,52,18.14,broken clouds
7,7,Butaritari,KI,3.0707,172.7902,83.61,74,13,14.07,few clouds
8,8,Hithadhoo,MV,-0.6000,73.0833,82.47,69,100,14.61,overcast clouds
13,13,Bubaque,GW,11.2833,-15.8333,81.54,75,21,5.06,few clouds
14,14,Freeport,US,40.6576,-73.5832,77.20,51,75,10.00,broken clouds
17,17,Puerto Escondido,MX,15.8500,-97.0667,85.87,68,100,6.55,overcast clouds
21,21,Hambantota,LK,6.1241,81.1185,79.68,82,74,17.54,broken clouds
25,25,Victoria,HK,22.2855,114.1577,85.32,82,14,3.00,light rain


In [8]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                203
City                   203
Country                202
Lat                    203
Lng                    203
Max Temp               203
Humidity               203
Cloudiness             203
Wind Speed             203
Current Description    203
dtype: int64

In [10]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df_clean = preferred_cities_df.dropna()
preferred_cities_df_clean.count()

City_ID                202
City                   202
Country                202
Lat                    202
Lng                    202
Max Temp               202
Humidity               202
Cloudiness             202
Wind Speed             202
Current Description    202
dtype: int64

In [11]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df_clean[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Kapaa,US,82.38,overcast clouds,22.0752,-159.3190,
2,Hilo,US,80.33,few clouds,19.7297,-155.0900,
5,Atuona,PF,77.49,broken clouds,-9.8000,-139.0333,
7,Butaritari,KI,83.61,few clouds,3.0707,172.7902,
8,Hithadhoo,MV,82.47,overcast clouds,-0.6000,73.0833,
13,Bubaque,GW,81.54,few clouds,11.2833,-15.8333,
14,Freeport,US,77.20,broken clouds,40.6576,-73.5832,
17,Puerto Escondido,MX,85.87,overcast clouds,15.8500,-97.0667,
21,Hambantota,LK,79.68,broken clouds,6.1241,81.1185,
25,Victoria,HK,85.32,light rain,22.2855,114.1577,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [15]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   202
Country                202
Max Temp               202
Current Description    202
Lat                    202
Lng                    202
Hotel Name             202
dtype: int64

In [16]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))